# Welcome to **PERfeCt** by AqADAPT team!
### To generate the interactive map and simulation graphical user interface, click on the *Restart the kernel, then re-run the whole notebook* button above or run the cells below consecutively using *Shift+Enter*.

**Loading the neccesary libraries***

In [ ]:
#%matplotlib inline
%matplotlib widget
from ipyleaflet import *
import ipywidgets as widgets
from ipywidgets import HTML, Layout, IntSlider, RadioButtons, Dropdown, Button, Output, Label
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import random
import json

# from deb.simulations import simulate_deb, plot_res, aux, fArr

**Variable setup**

In [ ]:
x, y = 0, 0
feature_clicked = False
name = None

with open('data/MED_farms/MED_farms_for_display.geojson', 'r') as f:
    data_farms = json.load(f)
with open('data/Natura2000_marine/Natura2000_marine_smooth.geojson', 'r') as f:
    data_Natura = json.load(f)
with open('data/MED_grid/MED_ter_10km_grid.geojson', 'r') as f:
    data_grid = json.load(f)

**Map setup**

In [ ]:
defaultLayout = Layout(width='1200px', height='500px')
center = (40,20)
icon = Icon(icon_url='', icon_size=[38, 95])
m = Map(center=center,
       basemap=basemaps.Esri.WorldImagery,
        zoom=4,
       layout=defaultLayout)

species_label = Label(layout=Layout(width="100%"))
title_label = HTML(value = "<p style='font-size:30px'>Welcome to PERfeCt!</p> <br> Select a location in the Mediterranean, then select a species and climate change scenario...")

def handle_map_click(**kwargs):
    if kwargs.get('type') == 'click':
        global x
        global y
        global dummy_num
        global feature_clicked
        global temp_marker
        
        try:
            m.remove_layer(temp_marker)
        except:
            pass
        
        if feature_clicked == True:
            temp_marker = Marker(
                    location=kwargs.get('coordinates'),
                    icon=icon,
                    name="Location info")           
            #print("You clicked on: ", kwargs.get('coordinates'))

            x, y = kwargs.get('coordinates')
            message = HTML()
            message.value = f"Coordinates: ({np.round(x,2)}, {np.round(y,2)})"
            species_label.value = f"Field: {name}"
            popup = Popup(
                location=(x, y),
                child=message,
                close_button=True,
                auto_close=True,
                close_on_escape_key=True,
                max_width = 500,
                name = "Location info"
            )
            m.add_layer(popup)
            m.add_layer(temp_marker)
        feature_clicked=False
        
m.on_interaction(handle_map_click)

geo_json_Natura = GeoJSON(
    data=data_Natura,
    style = {
        "fillOpacity": 0.75, "fillColor": "white", "color": "black", "weight": 1.5 
    },
    name="Natura2000 sites"
)

geo_json_grid = GeoJSON(
    data=data_grid,
    style = {
        "fillOpacity": 0.75, "fillColor": "#83d4e7", "color": "black", "weight": 0, "opacity": 0.5 
    },
    hover_style = {
        "fillOpacity": 0.75, "fillColor": "orange", "color": "white", "weight": 1, "opacity": 0.5 
    },
    name="Grid"
)

geo_json_farms = GeoJSON(
    data=data_farms,
    name="Farm locations"
)

def grid_click_handler(event=None, feature=None, id=None, properties=None):
    global name
    global feature_clicked
    
    feature_clicked = True

    try:
        m.remove_layer(temp_marker)
    except:
        pass
    
    name = properties["CellCode"]
    #species_label.value = f"Field: {name}"

def marker_click_handler(event=None, feature=None, id=None, properties=None):
    global name
    global feature_clicked
    global dummy_num
    global x
    global y

    try:
        m.remove_layer(temp_marker)
    except:
        pass
    #name = properties["geounit"]
    name = properties["species"]
    if name == None:
        name = properties["species 1"]
    x = properties["coordinates"][1]
    y = properties["coordinates"][0]
    species_label.value = f"Species: {name}"
    message = HTML()
    message.value = f"Coordinates: ({np.round(x,2)}, {np.round(y,2)}) <br> Species: {name[name.find('(')+1:name.find(')')]}"
    popup = Popup(
            location=(x, y),
            child=message,
            close_button=True,
            auto_close=True,
            close_on_escape_key=True,
            max_width = 500,
            name = "Location info"
        )
    m.add_layer(popup)
    
geo_json_farms.on_click(marker_click_handler)
geo_json_grid.on_click(grid_click_handler)

m.add_layer(geo_json_Natura)
m.add_layer(geo_json_grid)
m.add_layer(geo_json_farms)

control = LayersControl(position='topright')
m.add_control(control)

**Simulation GUI setup**

In [ ]:
display(title_label)
display(m)
# Print results
display(species_label)

options = [
    'Argyrosomus_regius',
    'Dentex_dentex',
    'Dicentrarchus_labrax',
    'Diplodus_puntazzo',
    'Diplodus_sargus',
    'Diplodus_vulgaris',
    'Lithognathus_mormyrus',
    'Oncorhynchus_mykiss',
    'Pagellus_erythrinus',
    'Salmo_salar',
    'Sarpa_salpa',
    'Sciaena_umbra',
    'Seriola_dumerili',
    'Sparus_aurata',
    'Thunnus_thynnus']

scenarios = ["RCP4.5", "RCP8.5"]

dropdown = Dropdown(
    options=options,
    value = options[0],
    description = "Species:",
    disabled = False)

radios = RadioButtons(
    options=scenarios,
    description="Scenario:",
    disabled=False
)

display(dropdown)
display(radios)

btn = Button(description="Simulate!")
out = Output()

display(btn)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
bar1 = ax[0].bar(np.arange(0, 30, 1), np.zeros(30))
bar2 = ax[1].bar(np.arange(0, 30, 1), np.zeros(30))
title1 = ax[0].set_title("Time to reach market size")
title2 = ax[1].set_title("Danger of vibriosis")
ax[0].set_ylim(0, 10)
ax[1].set_ylim(0, 10)
fig.suptitle("Simulation results")


def update(change):
    for i in range(30):
        bar1[i].set_height(np.random.randint(1, 10, 1))
        bar2[i].set_height(np.random.randint(1, 10, 1))
    ax[0].set_title(f"Time to market size {dropdown.value} - {radios.value}")
    ax[1].set_title(f"Danger of vibriosis - {radios.value}")
    fig.suptitle(f"Simulation results for location ({np.round(x,2)}, {np.round(y,2)})")
    fig.canvas.draw()
    return


#line1, = ax[0].plot(np.linspace(0,10), np.linspace(0,10))
#line2, = ax[1].plot(np.linspace(0,10), np.linspace(0,10))


def update1(change):
    line1.set_ydata(np.random.randint(0, 10, 50))
    line2.set_ydata(np.random.randint(0, 10, 50))
    ax[0].set_title(f"Time to market size {dropdown.value} - {radios.value}")
    ax[1].set_title(f"Danger of vibriosis - {radios.value}")
    fig.canvas.draw()
    return

#radios.observe(update, "value")

btn.on_click(update)